In [0]:
CREATE OR REFRESH STREAMING TABLE bronzee_races
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
       _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/ishwariformulabox/schemalanding/volume2/races.csv',
  'csv',
  map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_races (
  CONSTRAINT valid_race_id EXPECT (raceId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_year EXPECT (year IS NOT NULL AND year > 1900) ON VIOLATION DROP ROW,
  CONSTRAINT valid_round EXPECT (round IS NOT NULL AND round > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_circuit_id EXPECT (circuitId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_name EXPECT (name IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_date EXPECT (date IS NOT NULL AND date REGEXP '^\\d{2}-\\d{2}-\\d{4}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_time EXPECT (time IS NOT NULL AND time REGEXP '^\\d{2}:\\d{2}:\\d{2}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_url EXPECT (url IS NOT NULL AND url LIKE 'http%') ON VIOLATION DROP ROW,
  CONSTRAINT valid_fp1_date EXPECT (fp1_date IS NULL OR fp1_date REGEXP '^\\d{2}-\\d{2}-\\d{4}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_fp1_time EXPECT (fp1_time IS NULL OR fp1_time REGEXP '^\\d{2}:\\d{2}:\\d{2}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_fp2_date EXPECT (fp2_date IS NULL OR fp2_date REGEXP '^\\d{2}-\\d{2}-\\d{4}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_fp2_time EXPECT (fp2_time IS NULL OR fp2_time REGEXP '^\\d{2}:\\d{2}:\\d{2}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_fp3_date EXPECT (fp3_date IS NULL OR fp3_date REGEXP '^\\d{2}-\\d{2}-\\d{4}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_fp3_time EXPECT (fp3_time IS NULL OR fp3_time REGEXP '^\\d{2}:\\d{2}:\\d{2}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_quali_date EXPECT (quali_date IS NULL OR quali_date REGEXP '^\\d{2}-\\d{2}-\\d{4}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_quali_time EXPECT (quali_time IS NULL OR quali_time REGEXP '^\\d{2}:\\d{2}:\\d{2}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_sprint_date EXPECT (sprint_date IS NULL OR sprint_date REGEXP '^\\d{2}-\\d{2}-\\d{4}$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_sprint_time EXPECT (sprint_time IS NULL OR sprint_time REGEXP '^\\d{2}:\\d{2}:\\d{2}$') ON VIOLATION DROP ROW
)
COMMENT 'Cleaned race event data'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT raceId,
       year,
       round,
       circuitId,
       name,
       date,
       time,
       url,
       fp1_date,
       fp1_time,
       fp2_date,
       fp2_time,
       fp3_date,
       fp3_time,
       quali_date,
       quali_time,
       sprint_date,
       sprint_time
FROM STREAM(bronzee_races);
